In [ ]:
import pandas as pd  
import numpy as np
import sys

# Add parent directory to path for imports
sys.path.append('..')

from data.data_loder import GenotypeDataset,GenotypeDataset2,to_categorical,mask_data,process_data
from model.get_bv import get_bv
from model.pretrain_model import AutoEncoder
from train.train import train,test_bv,pre_train,train_bv
#  k_fold_train_with_train_func
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim

In [ ]:
genotype_file = './dataset/test_geno.csv'
phenotype_file = "./dataset/test_pheno.csv"

genotype = pd.read_csv(genotype_file,index_col=0)
genotype.index = pd.read_csv(genotype_file, usecols=[0]).iloc[:, 0]  

phenotype = pd.read_csv(phenotype_file, sep=',', index_col=0)
phenotype = phenotype.iloc[:, [2]]


In [ ]:
name_both= list(set(genotype.index).intersection(set(phenotype.index)))
genotype = genotype.loc[name_both,:]
phenotype = phenotype.loc[name_both,:]
ratio1=0
genotype = mask_data (genotype,ratio1)
phenotype = phenotype.values

In [21]:
train_loader_pre, valid_loader_pre, train_loader_bv, test_loader_bv = process_data(genotype=genotype,phenotype=phenotype,missing_perc=0)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
in_channels=3
n_snp=genotype.shape[-1]
model1 = AutoEncoder(in_channels=3, out_channels=256, kernel_size=40,stride=20, nhead=1, nhid=64, nlayers=1, dropout=0.25,device=device)
criterion_pre = nn.CrossEntropyLoss()
optimizer_pre = torch.optim.Adam(model1.parameters(), lr=1e-3)
model2 = get_bv( in_channels=in_channels, out_channels=256, kernel_size=5, stride=5, d_state=128,device=device, n_snp=n_snp)
criterion_bv = nn.MSELoss()
optimizer_bv = torch.optim.Adam(model2.parameters(), lr=0.00040854050495879857)

/public/tangly/anaconda3/envs/tang_scanpy/lib/python3.13/site-packages/torch/nn/modules/transformer.py:392: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


In [ ]:
epoch=100
model_bv_trained=train(epoch,model2,device,optimizer_bv,criterion_bv,train_loader_bv,test_loader_bv,pre_train_bool=False)

# model_bv_have_pre_trained=train(epoch,[model1,model2],
                                # device,[optimizer_pre,optimizer_bv],
                                # [criterion_pre,criterion_bv],
                                # train_loader_bv,
                                # test_loader_bv,pre_train_bool=True,
                                # train_loder_pre_train=train_loader_pre,
                                # test_loader_pre_train=1,
                                # epochs_pre_train=10)

Train Epoch: 100 [0/900 (0%)]	Loss: 2962.238770
Train Epoch: 100 [160/900 (18%)]	Loss: 442.868530
Train Epoch: 100 [320/900 (35%)]	Loss: 237.676636
Train Epoch: 100 [480/900 (53%)]	Loss: 66.145844
Train Epoch: 100 [640/900 (70%)]	Loss: 61.483185
Train Epoch: 100 [800/900 (88%)]	Loss: 50.637127
=========> Epoch: 100 Average loss: 233.5938
-0.14517884890794439
Train Epoch: 100 [0/900 (0%)]	Loss: 49.974667
Train Epoch: 100 [160/900 (18%)]	Loss: 60.092705
Train Epoch: 100 [320/900 (35%)]	Loss: 36.449780
Train Epoch: 100 [480/900 (53%)]	Loss: 33.811302
Train Epoch: 100 [640/900 (70%)]	Loss: 67.022675
Train Epoch: 100 [800/900 (88%)]	Loss: 72.686874
=========> Epoch: 100 Average loss: 277.7020
0.5702593522838594
Train Epoch: 100 [0/900 (0%)]	Loss: 45.724541
Train Epoch: 100 [160/900 (18%)]	Loss: 37.074471
Train Epoch: 100 [320/900 (35%)]	Loss: 24.171623
Train Epoch: 100 [480/900 (53%)]	Loss: 29.805168
Train Epoch: 100 [640/900 (70%)]	Loss: 46.725189
Train Epoch: 100 [800/900 (88%)]	Loss: 34.